# Imports

In [1]:
import requests
import json
import numpy as np
import pandas as pd
import os
import plotly.express as px
import datetime as datetime

# Definitions

In [2]:
cwd = os.getcwd()
leagueID = "1041760"
fpl_league_url = (
    f"https://fantasy.premierleague.com/api/leagues-classic/{leagueID}/standings/"
)
transfers_url_template = (
    "https://fantasy.premierleague.com/api/entry/{manager_id}/transfers/"
)
bootstrap_static_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

# Testing

## Main league df

In [3]:
response = requests.get(fpl_league_url)
if response.status_code == 200:
    print("The API request was successful.")
    fpl_league_response_json = response.json()
else:
    print(f"Error: The API request failed with status code {response.status_code}.")

The API request was successful.


In [4]:
league_df = pd.DataFrame(fpl_league_response_json["standings"]["results"])
league_df

,id,event_total,player_name,rank,last_rank,rank_sort,total,entry,entry_name
0,35832424,80,James Parfitt,1,1,1,1396,4651100,Where you been?
1,35939248,72,Alex Winn,2,3,2,1346,3794145,Al-Winn
2,58581266,49,Seb Winn,3,2,3,1340,7052472,Gangsters Allardyce
3,35832433,49,Ben Melhuish,4,6,4,1263,4075412,Melshys Mediocres
4,35879230,53,Joseph Parsons,5,8,5,1255,4656770,AFC Great Name UTD
5,56846765,57,Richard Collins,6,9,6,1253,6911033,Eze-er said than Son
6,35832439,24,Jacob Pickles,7,4,7,1249,3148659,Gobblebox
7,57395169,21,Mason Gardener,8,5,8,1235,6963106,nice to Michu
8,35832437,31,Dan Pratlett,9,7,9,1235,3194123,Onana what’s my name
9,48472859,25,Ben Wilson,10,10,10,1139,6083206,Kloppenheimer


## Open player data and make ID map

In [5]:
players_df = pd.read_csv(cwd + "/../data/app/players_raw.csv")
players_df.loc[:, ["id", "web_name"]]
id_name_dict = pd.Series(
    players_df["web_name"].values, index=players_df["id"]
).to_dict()
id_name_dict

{1: 'Balogun',
 2: 'Cédric',
 3: 'M.Elneny',
 4: 'Fábio Vieira',
 5: 'Gabriel',
 6: 'Havertz',
 8: 'G.Jesus',
 9: 'Jorginho',
 10: 'Kiwior',
 11: 'Marquinhos',
 12: 'Martinelli',
 13: 'Nketiah',
 14: 'Ødegaard',
 15: 'Thomas',
 16: 'Pepe',
 17: 'Ramsdale',
 18: 'Rúnarsson',
 19: 'Saka',
 20: 'Saliba',
 22: 'Smith Rowe',
 24: 'Tierney',
 25: 'Tomiyasu',
 26: 'Trossard',
 29: 'White',
 30: 'Xhaka',
 31: 'Zinchenko',
 113: 'Raya',
 540: 'Rice',
 578: 'Nelson',
 585: 'J.Timber',
 646: 'Hein',
 735: 'Sagoe',
 736: 'Walters',
 737: 'Ibrahim',
 764: 'Lewis-Skelly',
 772: 'Nwaneri',
 32: 'Alex Moreno',
 34: 'Bailey',
 35: 'Buendia',
 36: 'Cash',
 37: 'Chambers',
 38: 'Coutinho',
 39: 'Davis',
 40: 'Dendoncker',
 41: 'Diego Carlos',
 42: 'Digne',
 43: 'Douglas Luiz',
 44: 'Duran',
 45: 'Hause',
 46: 'Iroegbunam',
 47: 'Kamara',
 48: 'Konsa',
 49: 'Martinez',
 50: 'McGinn',
 51: 'Mings',
 53: 'Olsen',
 54: 'Philogene',
 55: 'J.Ramsey',
 56: 'Sanson',
 57: 'Sinisalo',
 58: 'Tielemans',
 59: 'Bert

## Do the same for manager_id

In [6]:
manager_id_name_dict = pd.Series(
    league_df["player_name"].values, index=league_df["entry"]
).to_dict()
manager_id_name_dict

{4651100: 'James Parfitt',
 3794145: 'Alex Winn',
 7052472: 'Seb Winn',
 4075412: 'Ben Melhuish',
 4656770: 'Joseph Parsons',
 6911033: 'Richard Collins',
 3148659: 'Jacob Pickles',
 6963106: 'Mason Gardener',
 3194123: 'Dan Pratlett',
 6083206: 'Ben Wilson',
 2709514: 'Ben Anderson',
 4371621: 'Luke Counsell'}

## Get tranfers df

In [7]:
def human_readable(num):
    if num > 1000000:
        if not num % 1000000:
            return f"{num // 1000000}M"
        return f"{round(num / 1000000, 1)}M"
    return f"{num // 1000}K"


transfers_dfs_list = []
for manager_id in league_df["entry"].values:
    response = requests.get(transfers_url_template.format(manager_id=manager_id))
    transfers_response_json = response.json()
    if not transfers_response_json:
        continue
    transfers_df = pd.DataFrame(transfers_response_json)
    transfers_df["element_in"] = transfers_df["element_in"].map(id_name_dict)
    transfers_df["element_in_cost"] = (transfers_df["element_in_cost"] * 1e5).apply(
        lambda n: human_readable(n)
    )
    transfers_df["element_out"] = transfers_df["element_out"].map(id_name_dict)
    transfers_df["element_out_cost"] = (transfers_df["element_out_cost"] * 1e5).apply(
        lambda n: human_readable(n)
    )
    transfers_df["manager_id"] = manager_id
    transfers_df["Manager"] = manager_id_name_dict[manager_id]
    # transfers_df["time"] = pd.to_datetime(transfers_df["time"]).dt.round("1s")
    transfers_dfs_list.append(transfers_df)
all_managers_transfers_df = pd.concat(transfers_dfs_list)
all_managers_transfers_df.dtypes

element_in          object
element_in_cost     object
element_out         object
element_out_cost    object
entry                int64
event                int64
time                object
manager_id           int64
Manager             object
dtype: object

In [8]:
all_managers_transfers_df.loc[all_managers_transfers_df["Manager"] == "Richard Collins", :]

,element_in,element_in_cost,element_out,element_out_cost,entry,event,time,manager_id,Manager
0,Gordon,6.2M,Douglas Luiz,5.6M,6911033,23,2024-02-03T09:09:18.238723Z,6911033,Richard Collins
1,Haaland,14.1M,J.Alvarez,6.9M,6911033,23,2024-02-03T09:09:18.232316Z,6911033,Richard Collins
2,Solanke,7.1M,Archer,4.5M,6911033,21,2024-01-12T14:41:34.670567Z,6911033,Richard Collins
3,Douglas Luiz,5.6M,Sarr,4.5M,6911033,21,2024-01-12T14:41:34.670338Z,6911033,Richard Collins
4,Richarlison,6.9M,Son,9.6M,6911033,21,2024-01-12T14:41:34.670118Z,6911033,Richard Collins
5,Saka,9.1M,Salah,12.8M,6911033,21,2024-01-12T14:41:34.669862Z,6911033,Richard Collins
6,Palmer,5.7M,Hee Chan,5.7M,6911033,21,2024-01-12T14:41:34.669626Z,6911033,Richard Collins
7,Foden,7.9M,Bowen,7.9M,6911033,21,2024-01-12T14:41:34.669389Z,6911033,Richard Collins
8,Virgil,6.3M,Alexander-Arnold,8.4M,6911033,21,2024-01-12T14:41:34.669138Z,6911033,Richard Collins
9,Areola,4.2M,Sanchez,4.6M,6911033,21,2024-01-12T14:41:34.668804Z,6911033,Richard Collins


## Get GW dealines

In [13]:
response = requests.get(bootstrap_static_url)
bootstrap_static_response = response.json()
bootstrap_static_df = pd.DataFrame(bootstrap_static_response["events"])
bootstrap_static_df.head(-10)

,id,name,deadline_time,release_time,average_entry_score,finished,data_checked,highest_scoring_entry,deadline_time_epoch,deadline_time_game_offset,...,h2h_ko_matches_created,ranked_count,chip_plays,most_selected,most_transferred_in,top_element,top_element_info,transfers_made,most_captained,most_vice_captained
0,1,Gameweek 1,2023-08-11T17:30:00Z,None,64,True,True,3383750.0,1691775000,0,...,False,0,"[{'chip_name': 'bboost', 'num_played': 163222}...",355.0,1.0,395.0,"{'id': 395, 'points': 14}",0,355.0,19.0
1,2,Gameweek 2,2023-08-18T17:15:00Z,None,44,True,True,3338487.0,1692378900,0,...,True,0,"[{'chip_name': 'bboost', 'num_played': 126778}...",355.0,195.0,108.0,"{'id': 108, 'points': 16}",13130353,355.0,19.0
2,3,Gameweek 3,2023-08-25T17:30:00Z,None,44,True,True,9368956.0,1692984600,0,...,True,0,"[{'chip_name': 'bboost', 'num_played': 124110}...",355.0,108.0,216.0,"{'id': 216, 'points': 19}",17619532,355.0,19.0
3,4,Gameweek 4,2023-09-01T17:30:00Z,None,72,True,True,4354697.0,1693589400,0,...,True,0,"[{'chip_name': 'bboost', 'num_played': 109196}...",355.0,216.0,516.0,"{'id': 516, 'points': 20}",16035365,355.0,19.0
4,5,Gameweek 5,2023-09-16T10:00:00Z,None,44,True,True,6211182.0,1694858400,0,...,True,0,"[{'chip_name': 'bboost', 'num_played': 96469},...",355.0,516.0,344.0,"{'id': 344, 'points': 13}",14363988,355.0,19.0
5,6,Gameweek 6,2023-09-23T12:30:00Z,None,68,True,True,4494759.0,1695472200,0,...,True,0,"[{'chip_name': 'bboost', 'num_played': 78528},...",355.0,343.0,430.0,"{'id': 430, 'points': 18}",12109066,355.0,308.0
6,7,Gameweek 7,2023-09-30T10:00:00Z,None,49,True,True,9335002.0,1696068000,0,...,True,0,"[{'chip_name': 'bboost', 'num_played': 79672},...",355.0,430.0,60.0,"{'id': 60, 'points': 23}",15579302,355.0,308.0
7,8,Gameweek 8,2023-10-07T10:00:00Z,None,44,True,True,10068163.0,1696672800,0,...,True,0,"[{'chip_name': 'bboost', 'num_played': 53401},...",355.0,516.0,216.0,"{'id': 216, 'points': 16}",19444885,355.0,355.0
8,9,Gameweek 9,2023-10-21T10:00:00Z,None,67,True,True,6931177.0,1697882400,0,...,True,0,"[{'chip_name': 'bboost', 'num_played': 50175},...",355.0,60.0,423.0,"{'id': 423, 'points': 17}",11431916,355.0,308.0
9,10,Gameweek 10,2023-10-27T17:30:00Z,None,66,True,True,9895223.0,1698427800,0,...,True,0,"[{'chip_name': 'bboost', 'num_played': 80903},...",355.0,60.0,13.0,"{'id': 13, 'points': 17}",12428517,355.0,355.0


# Plot

In [10]:
fig = (
    px.scatter(
        all_managers_transfers_df,
        x="time",
        y="Manager",
        color="Manager",
        hover_name=None,
        hover_data={
            "Manager": True,
            "time": False,
            "element_in": True,
            "element_in_cost": True,
            "element_out": True,
            "element_out_cost": True,
            "event": False,
        },
    )
    .update_xaxes(rangeslider_visible=True, range=["2024-01-29", "2024-02-05"])
    .update_layout(showlegend=True, yaxis_type="category", hovermode="x unified")
)
for gw in range(1, 39):
    fig.add_vline(
        x=datetime.datetime.strptime(
            bootstrap_static_df.loc[gw - 1, "deadline_time"], "%Y-%m-%dT%H:%M:%SZ" # type: ignore
        ).timestamp()
        * 1000,
        line_width=1,
        line_dash="dash",
        line_color="red",
        annotation_text=f"Gameweek {gw} Deadline",
        annotation_position="top",
    )
fig.show()